In [ ]:
# IMD stats for Lewisham.
# Loads IMD data and prepares it for use in Lewisham ward profiles.
#
# This will not be available at Ward level, however we can annotate 
# which LSOAs to show for which ward.

In [1]:
import numpy as np
import pandas as pd

from google.colab import drive
import google.colab.files as files

import IPython

Data
==
GDrive mount
--

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Used as root folder.
project_dir = '/content/gdrive/MyDrive/WardProfiles'
ward_lookups_dir = f"{project_dir}/lookups/2022_wards"

# For exports
out_dir = f"{project_dir}/deprivation"

In [ ]:
!mkdir -p '{out_dir}'

Downloads
--

In [ ]:
# 2019 IMD, at LSOA level
# Source: 
# https://www.gov.uk/government/statistics/english-indices-of-deprivation-2019
!wget 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833970/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx' \
  -O "{out_dir}/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx"

--2021-11-04 12:56:11--  https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833970/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx
Resolving assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)... 151.101.0.144, 151.101.64.144, 151.101.128.144, ...
Connecting to assets.publishing.service.gov.uk (assets.publishing.service.gov.uk)|151.101.0.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1433347 (1.4M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘/content/gdrive/MyDrive/WardProfiles/deprivation/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx’

/content/gdrive/MyD 100%[===================>]   1.37M  --.-KB/s    in 0.1s    

2021-11-04 12:56:13 (13.3 MB/s) - ‘/content/gdrive/MyDrive/WardProfiles/deprivation/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx’ saved [1433347/1433347]



In [ ]:
!ls -lh "{out_dir}"

total 1.4M
-rw------- 1 root root 1.4M Sep 25  2019 File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx


Lookups
--
Used to match up LSOAs with 2022 electoral wards

In [4]:
# All LSOAs and their 2022 wards in Lewisham
lsoa_wd22_join = pd.read_csv(f"{ward_lookups_dir}/lbl_lsoa11_wd22_proposed.csv")
lsoa_wd22_join = lsoa_wd22_join[['LSOA11CD', 'WD22CD_proposed']]
lsoa_wd22_join.head()

,LSOA11CD,WD22CD_proposed
0,E01003316,E05013729
1,E01003314,E05013729
2,E01003312,E05013729
3,E01003313,E05013729
4,E01003310,E05013729


Process & export
==

In [5]:
# For filtering of source data
lbl_lsoa_list = lsoa_wd22_join.LSOA11CD.unique() # All LSOAs in Lewisham
len(lbl_lsoa_list)

169

In [6]:
# Source data
d = pd.read_excel(f"{out_dir}/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx", 
                  sheet_name='IMD2019',
                  #skiprows=7, # Data starts in row 8
                  #skipfooter=5
                  ) # Data has some footer rows
d.head()

,LSOA code (2011),LSOA name (2011),Local Authority District code (2019),Local Authority District name (2019),Index of Multiple Deprivation (IMD) Rank,Index of Multiple Deprivation (IMD) Decile
0,E01000001,City of London 001A,E09000001,City of London,29199,9
1,E01000002,City of London 001B,E09000001,City of London,30379,10
2,E01000003,City of London 001C,E09000001,City of London,14915,5
3,E01000005,City of London 001E,E09000001,City of London,8678,3
4,E01000006,Barking and Dagenham 016A,E09000002,Barking and Dagenham,14486,5


In [7]:
# Standardise column names
d = d.rename(columns={
    'LSOA code (2011)': 'LSOA11CD', 
    'LSOA name (2011)': 'LSOA11NM',
    'Local Authority District code (2019)': 'LAD19CD',
       'Local Authority District name (2019)': 'LAD19NM',
       'Index of Multiple Deprivation (IMD) Rank': 'IMD_rank',
       'Index of Multiple Deprivation (IMD) Decile': 'IMD_decile'
})
d.columns

Index(['LSOA11CD', 'LSOA11NM', 'LAD19CD', 'LAD19NM', 'IMD_rank', 'IMD_decile'], dtype='object')

In [8]:
# Limit to Lewisham LSOAs
lbl_imd = d[d.LSOA11CD.isin(lbl_lsoa_list)].copy()

# Add Ward codes
lsoa_wd22_map = lsoa_wd22_join.set_index('LSOA11CD').WD22CD_proposed.to_dict()
lbl_imd['WD22CD_proposed'] = lbl_imd.LSOA11CD.map(lambda lsoa: lsoa_wd22_map[lsoa])

In [ ]:
# Export
lbl_imd.to_csv(f"{out_dir}/lbl_imd_lsoa11.csv", index=False)
IPython.display.display(len(lbl_imd))
IPython.display.display(lbl_imd.head())

169

,LSOA11CD,LSOA11NM,LAD19CD,LAD19NM,IMD_rank,IMD_decile,WD22CD_proposed
3121,E01003189,Lewisham 034A,E09000023,Lewisham,3400,2,E05013714
3122,E01003190,Lewisham 034B,E09000023,Lewisham,5993,2,E05013714
3123,E01003191,Lewisham 034C,E09000023,Lewisham,5923,2,E05013714
3124,E01003192,Lewisham 034D,E09000023,Lewisham,2371,1,E05013714
3125,E01003193,Lewisham 029A,E09000023,Lewisham,7450,3,E05013729


Results
--

In [ ]:
!ls -lh '{out_dir}'

total 1.4M
-rw------- 1 root root 1.4M Sep 25  2019 File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx
-rw------- 1 root root  11K Nov  4 13:09 lbl_imd_lsoa11.csv


In [ ]:
# !rm '{out_dir}'/*.csv